# PA!(Presentation Assistant) 서비스
*해당 코드는 노트북 상에서 작동되는 서비스로 본 서비스와 코드는 [PA!(Presentation Assistdant)](https://huggingface.co/spaces/SRankChatGpt/Presentation-Assistant)에서 체험하실 수 있습니다* 😀
> **파!🥬 PA! (Presentation Assistant)**는 PPT 제작 및 프레젠테이션 지원 Text2PPT & PPT2Script 플랫폼입니다.    
사용자가 입력한 내용을 기반으로 PPT를 제작하고, 프레젠테이션 스크립트를 제공하여 프레젠테이션 역량을 향상시킵니다.    
자동화된 PPT 제작 기능을 통해 업무 효율성을 높이고, 시간과 비용을 절약할 수 있습니다. 

- 서비스 관련 문의는 Github Issue 및 [Huggingface Community](https://huggingface.co/spaces/SRankChatGpt/Presentation-Assistant/discussions)로 부탁드립니다.
    - 피드백 및 버그 신고는 언제나 환영입니다 🙂

## 1. 환경 설정
- 라이브러리 설치 및 임포트
- 한글 폰트 설치(나눔고딕)

In [ ]:
# 파이썬 라이브러리 설치
!pip install -q openai
!pip install PyPDF2
!pip install python-pptx==0.6.18

# pandoc
!sudo apt-get install pandoc

# marp
!pip install nodejs-bin
!curl https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb -o chrome.deb
!dpkg --install chrome.deb
!apt-get install --fix-broken --assume-yes
!npm install -g @marp-team/marp-cli

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
pandoc is already the newest version (2.5-3build2).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 MB 14.2 MB/s eta 0:00:00
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 89.6M  100 89.6M    0     0  4340k      0  0:00:21  0:00:21 --:--:-- 4764k
(Reading database ... 122604 files and directories currently installed.)
Preparing to unpack chrome.deb ...
Unpacking google-chrome-stable (112.0.5615.165-1) over (112.0.561

In [ ]:
# 폰트 설치
!sudo apt-get install -f
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
!fc-list | grep "Nanum"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 23 not upgraded.
Need to get 9,599 kB of archives.
After this operation, 29.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 fonts-nanum all 20180306-3 [9,599 kB]
Fetched 9,599 kB in 2s (5,544 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back t

In [ ]:
# 라이브러리 불러오기
import subprocess
import PyPDF2
import openai
from pptx import Presentation
from io import BytesIO
from nodejs import npx

In [ ]:
# API KEY 입력
# https://platform.openai.com/account/api-keys 에서 확인 가능
openapi_key = 'API를 입력해주세요!' 

## 2. TEXT2PPT

### 2-1. Marp 프롬프트 생성

In [ ]:
def generate_text2ppt_input_prompt_marp(input_type, input_value, input_pages):
    header="""
    너가 Marp 문법으로 PPT를 제작하는 디자이너라고 가정하고, %s장의 PPT를 작성해.
    +++ 아래 내용 또는 링크를 요약해서 마크다운 언어로 작성하는데, === 아래 규칙과 지키고, ~~~ 아래 슬라이드 예시를 참고해.

    +++
    """%input_pages

    summary_value = ""

    if input_type == "링크":
        summary_value += input_value
        summary_value += "\n"
    elif input_type == "텍스트":
        summary_value += input_value
        summary_value += "\n"
    elif input_type == "PDF":
        with open(input_value, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            num_pages = len(pdf_reader.pages)

        # 각 페이지의 내용을 문자열로 변환합니다.
        text = ""
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
        summary_value += text
        summary_value += "\n"
    else:
        print("ERROR: 잘못된 입력")

    rule_value = """
    ===
    - 제시한 내용 또는 링크의 내용에 대해서만 사실적으로 작성해줘.
    - 슬라이드 구분자로 ---를 무조건 사용해줘.
    - 주제에 적절한 도형, 이미지(![이미지](이미지링크), https://unsplash.com/ko/images/stock/non-copyrighted 에서 실제로 사용 가능한), 표(|-|), 인용(>), 강조(bold, ``), 이모지(https://kr.piliapp.com/twitter-symbols/), 아이콘 (https://kr.piliapp.com/symbol/#popular) 등이 다양하게 슬라이드를 디자인하고 배치해줘.
    - 이모지는 최대 2 페이지에 한 번만 사용하고, 다른 디자인을 다양하게 사용해줘.
    - 이미지와 표를 사용할 때, 페이지 크기와 고려해서 글 내용이 모두 나타나도록 크기를 지정해줘.
    - Slide 1를 제목으로 해서 총 %s장이야.
    - PPT의 내용을 풍부하게 마크다운으로 작성해줘.
    - 슬라이드 별로 설명하지말고, 코드만 작성해줘.
    - 예시의 내용을 사용해서 작성하지말고, 형식만 참고해.

    ~~~
    <!-- Slide 0. 슬라이드 주제 -->
    # 슬라이드 제목
    ![이미지링크](https://huggingface.co/datasets/huggingface/brand-assets/resolve/main/hf-logo-with-title.png)
    - ChatGPT를 활용한 🤗**TEXT2PPT 서비스 PA!**입니다.
    - `링크`,`텍스트`, `PDF`를 입력 또는 업로드하면, PPT로 변환합니다. 
    """%input_pages

    return header+summary_value+rule_value

### 2-2. Pandoc 프롬프트 생성

In [ ]:
def generate_text2ppt_input_prompt_pandoc(input_type, input_value, input_pages):
    header = """
    너가 markdown 문법으로 PPT를 제작하는 디자이너라고 가정하고, %s장의 PPT를 작성해.
    +++ 아래 내용 또는 링크를 요약해서 markdown 언어로 작성하는데, === 아래 규칙과 지키고, ~~~ 아래 슬라이드 예시를 참고해.
    +++
    """ % input_pages

    summary_value = ""

    if input_type == "링크":
        summary_value += input_value
        summary_value += "\n"
    elif input_type == "텍스트":
        summary_value += input_value
        summary_value += "\n"
    elif input_type == "PDF":
        with open(input_value, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            num_pages = len(pdf_reader.pages)

        # 각 페이지의 내용을 문자열로 변환합니다.
        text = ""
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
        summary_value += text
        summary_value += "\n"
    else:
        print("ERROR: 잘못된 입력")

    rule_value = """
    ===
    - 제시한 내용 또는 링크의 내용에 대해서만 사실적으로 작성해줘.
    - 슬라이드 구분자로 ---를 무조건 사용해줘.
    - 주제에 적절한 도형, 이미지(![이미지](이미지링크), https://unsplash.com/ko/images/stock/non-copyrighted 에서 실제로 사용 가능한), 표(|-|), 인용(>), 강조(bold, ``), 이모지(https://kr.piliapp.com/twitter-symbols/), 아이콘 (https://kr.piliapp.com/symbol/#popular) 등이 다양하게 슬라이드를 디자인하고 배치해줘.
    - 이모지는 최대 2 페이지에 한 번만 사용하고, 다른 디자인을 다양하게 사용해줘.
    - 이미지와 표를 사용할 때, 페이지 크기와 고려해서 글 내용이 모두 나타나도록 크기를 지정해줘.
    - Slide 1를 제목으로 해서 총 %s장이야.
    - PPT의 내용을 풍부하게 마크다운으로 작성해줘.
    - 슬라이드 별로 설명하지말고, 코드만 작성해줘.
    - 예시의 내용을 사용해서 작성하지말고, 형식만 참고해.
    ~~~
    <!-- Slide 0. 슬라이드 주제 -->
    # 슬라이드 제목
    ![이미지링크](https://huggingface.co/datasets/huggingface/brand-assets/resolve/main/hf-logo-with-title.png)
    - ChatGPT를 활용한 🤗**TEXT2PPT 서비스 PA!**입니다.
    - `링크`,`텍스트`, `PDF`를 입력 또는 업로드하면, PPT로 변환합니다. 
    """ % input_pages

    return header + summary_value + rule_value

### 2-3. TEXT2PPT 실행 함수

In [ ]:
def text2ppt(token_key, input_prompt, input_theme, trans_type):
    openai.api_key = token_key

    messages = [
        {"role": "system", "content": "You are a kind helpful PPT designer."},
    ]

    message = input_prompt

    if message:
        messages.append(
            {"role": "user", "content": message},
        )
        chat = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0301", messages=messages
        )

    if trans_type == 'pandoc':
        reply = chat.choices[0].message.content
        messages.append({"role": "assistant", "content": reply})

        md_text = reply[4:] if reply[:3] == "---" else reply
        md_text_list = md_text.split('\n')

        f = open("text2ppt_input.md", 'w')
        for i in range(0, len(md_text_list)):
            data = md_text_list[i] + "\n"
            f.write(data)
        f.close()


        if input_theme == 'default':
            subprocess.run(["pandoc", "text2ppt_input.md", "-t", "pptx", "-o", "text2ppt_output.pptx"], capture_output=True)
        else:
            ppt_theme = "--reference-doc="+input_theme+".pptx"
            subprocess.run(["pandoc", "text2ppt_input.md", "-t", "pptx", ppt_theme, "-o", "text2ppt_output.pptx"], capture_output=True)
    else:
        reply = chat.choices[0].message.content
        messages.append({"role": "assistant", "content": reply})

        revised_reply = reply[4:] if reply[:3]=="---" else reply

        ppt_theme = input_theme

        marp_header="""---
        marp: true
        theme: %s
        footer: 'PA!(Presentation Assistant)'
        paginate: true
        ---
        """%input_theme

        md_text=marp_header+revised_reply

        md_text_list = md_text.split('\n')

        f = open("text2ppt_input_marp.md", 'w')
        for i in range(0, len(md_text_list)):
            data = md_text_list[i] + "\n"
            f.write(data)
        f.close()

        npx.run(["@marp-team/marp-cli@latest", "--pdf-fonts-dir", "/usr/share/fonts/truetype/nanum", "--pdf-default-font", "NanumGothic", "-o", "text2ppt_output_marp.pptx", "text2ppt_input_marp.md", "--chrome-path", "/usr/bin/google-chrome-stable"], capture_output=True)

## 3. PPT2Script

In [ ]:
def ppt2script(token_key, input_file, input_type):
    openai.api_key = token_key

    if input_type=="PDF":
        with open(input_file, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            num_pages = len(pdf_reader.pages)
    
            text = ""
            for page_num in range(num_pages):
                page = pdf_reader.pages[page_num]
                text += "[PAGE_NUM " + str(page_num + 1) + "]"
                text += page.extract_text()
    else:
        prs = Presentation(input_file)

        text = ""
        page_num = 0
        for slide in prs.slides:
            text += "[PAGE_NUM " + str(page_num + 1) + "]"
            page_num += 1
            for shape in slide.shapes:
                if not shape.has_text_frame:
                    continue
                for paragraph in shape.text_frame.paragraphs:
                    for run in paragraph.runs:
                        text += run.text

    header = """
    너는는 PPT 발표에 도움을 주는 조력자야. 
    ~~~아래 규칙을 지키고, --- 아래 PPT 내용에 대해 발표 스크립트를 한글로 작성해.
    ~~~
    - [PAGE_NUM 1] 일 때, 여기서 1은 페이지 번호인데, 페이지 번호마다 발표 스크립트를 작성해.
    - 마크다운 언어를 쓰지않고, 텍스트로만 작성해.
    - PPT 내용에 추가적인 설명이나 사례를 덧붙여줘.
    ---
    """

    input_prompt = header + text

    messages = [
        {"role": "system", "content": "You are a kind helpful PPT Assistant."},
    ]

    message = input_prompt

    if message:
        messages.append(
            {"role": "user", "content": message},
        )
        chat = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0301", messages=messages
        )

    reply = chat.choices[0].message.content
    messages.append({"role": "assistant", "content": reply})

    return reply

## 4. TEXT2PPT 서비스

### 4-1. 입력 문서 작성

In [ ]:
input_text = input("텍스트 또는 링크를 입력해주세요!")

텍스트 또는 링크를 입력해주세요!https://ko.wikipedia.org/wiki/수소


### 4-2. MARP 기반 TEXT2PPT

#### 4-2-1. 입력 프롬프트 생성

In [ ]:
# PPT 희망 장수 지정
input_pages = 5

In [ ]:
input_prompt_marp = generate_text2ppt_input_prompt_marp("텍스트", input_text, input_pages)

In [ ]:
print(input_prompt_marp)


    너가 Marp 문법으로 PPT를 제작하는 디자이너라고 가정하고, 5장의 PPT를 작성해.
    +++ 아래 내용 또는 링크를 요약해서 마크다운 언어로 작성하는데, === 아래 규칙과 지키고, ~~~ 아래 슬라이드 예시를 참고해.

    +++
    https://ko.wikipedia.org/wiki/수소

    ===
    - 제시한 내용 또는 링크의 내용에 대해서만 사실적으로 작성해줘.
    - 슬라이드 구분자로 ---를 무조건 사용해줘.
    - 주제에 적절한 도형, 이미지(![이미지](이미지링크), https://unsplash.com/ko/images/stock/non-copyrighted 에서 실제로 사용 가능한), 표(|-|), 인용(>), 강조(bold, ``), 이모지(https://kr.piliapp.com/twitter-symbols/), 아이콘 (https://kr.piliapp.com/symbol/#popular) 등이 다양하게 슬라이드를 디자인하고 배치해줘.
    - 이모지는 최대 2 페이지에 한 번만 사용하고, 다른 디자인을 다양하게 사용해줘.
    - 이미지와 표를 사용할 때, 페이지 크기와 고려해서 글 내용이 모두 나타나도록 크기를 지정해줘.
    - Slide 1를 제목으로 해서 총 5장이야.
    - PPT의 내용을 풍부하게 마크다운으로 작성해줘.
    - 슬라이드 별로 설명하지말고, 코드만 작성해줘.
    - 예시의 내용을 사용해서 작성하지말고, 형식만 참고해.

    ~~~
    <!-- Slide 0. 슬라이드 주제 -->
    # 슬라이드 제목
    ![이미지링크](https://huggingface.co/datasets/huggingface/brand-assets/resolve/main/hf-logo-with-title.png)
    - ChatGPT를 활용한 🤗**TEXT2PPT 서비스 PA!**입니다.
    - `링크`,`텍스트`, `PDF`를 입력 또는 업로드하면, PP

#### 4-2-2. PPT 변환
- 좌측 파일 시스템에서 생성된 파일을 확인하실 수 있습니다.
- 파일명: `text2ppt_output_marp.pptx`

In [ ]:
# MARP 변환 테마 지정
input_theme = "default" # gaia, uncover 등 다양한 테마가 있습니다.

In [ ]:
text2ppt(openapi_key, input_prompt_marp, input_theme, "MARP") # 문서 길이 및 PPT 장수에 따라 최소 2~10분 소요됩니다.

In [ ]:
npx.run(["@marp-team/marp-cli@latest", "--pdf-fonts-dir", "/usr/share/fonts/truetype/nanum", "--pdf-default-font", "NanumGothic", "-o", "text2ppt_output_marp.pptx", "text2ppt_input_marp.md", "--chrome-path", "/usr/bin/google-chrome-stable"], capture_output=True)

CompletedProcess(args=['/usr/local/lib/python3.9/dist-packages/nodejs/bin/node', '/usr/local/lib/python3.9/dist-packages/nodejs/lib/node_modules/npm/bin/npx-cli.js', '@marp-team/marp-cli@latest', '--pdf-fonts-dir', '/usr/share/fonts/truetype/nanum', '--pdf-default-font', 'NanumGothic', '-o', 'text2ppt_output_marp.pptx', 'text2ppt_input_marp.md', '--chrome-path', '/usr/bin/google-chrome-stable'], returncode=0, stdout=b'', stderr=b'[  INFO ] Converting 1 markdown...\n[  INFO ] text2ppt_input_marp.md => text2ppt_output_marp.pptx\n')

In [ ]:
# pdf 변환을 희망하시면 아래 코드를 사용하시면 됩니다.
npx.run(["@marp-team/marp-cli@latest", "--pdf-fonts-dir", "/usr/share/fonts/truetype/nanum", "--pdf-default-font", "NanumGothic", "-o", "text2ppt_output_marp.pdf", "text2ppt_input_marp.md", "--chrome-path", "/usr/bin/google-chrome-stable"], capture_output=True)

CompletedProcess(args=['/usr/local/lib/python3.9/dist-packages/nodejs/bin/node', '/usr/local/lib/python3.9/dist-packages/nodejs/lib/node_modules/npm/bin/npx-cli.js', '@marp-team/marp-cli@latest', '--pdf-fonts-dir', '/usr/share/fonts/truetype/nanum', '--pdf-default-font', 'NanumGothic', '-o', 'text2ppt_output_marp.pdf', 'text2ppt_input_marp.md', '--chrome-path', '/usr/bin/google-chrome-stable'], returncode=0, stdout=b'', stderr=b'[  INFO ] Converting 1 markdown...\n[  INFO ] text2ppt_input_marp.md => text2ppt_output_marp.pdf\n')

### 4-3. Pandoc 기반 TEXT2PPT

#### 4-3-1. 입력 프롬프트 생성

In [ ]:
# PPT 희망 장수 지정
input_pages = 5

In [ ]:
input_prompt_pandoc = generate_text2ppt_input_prompt_pandoc("텍스트", input_text, input_pages)

In [ ]:
# Pandoc 변환 테마 지정
input_theme = "default"

# CUSTOM 테마는 아래 코드를 참고하세요!
# input_theme = "custom" # 파일 이름을 작성해주세요 (.pptx 제외) ex) blue.pptx -> blue만 입력
# from google.colab import files
# myfiles = files.upload()

#### 4-3-2. PPT 변환
- 좌측 파일 시스템에서 생성된 파일을 확인하실 수 있습니다.
- 파일명: `text2ppt_output.pptx`
- PPT 다운로드 후 편집 가능

In [ ]:
text2ppt(openapi_key, input_prompt_marp, input_theme, "pandoc") # 문서 길이 및 PPT 장수에 따라 최소 2~10분 소요됩니다.

## 5. PPT2Script 서비스

#### 5-1. 파일 업로드

In [ ]:
from google.colab import files

input_type = 'PPT' # PDF 파일의 경우 'PDF'로 수정하시면 됩니다.
file_name = 'ppt2script_input.pptx' # 업로드한 파일에 맞춰서 이름을 변경해주세요!
my_file = files.upload()

Saving ppt2script_input.pptx to ppt2script_input (1).pptx


### 5-2. 스크립트 변환

In [ ]:
script = ppt2script(openapi_key, file_name, input_type) # PPT 또는 PDF 크기에 따라 최소 2~10분 소요됩니다.

In [ ]:
print(script)

[PAGE_NUM 1] 수소란, 원자 번호 1(H)을 가진 산소, 탄소와 더불어 자연에서 가장 흔하게 존재하는 화학 원소 중 하나입니다. 수소는 가장 간단한 구조를 가지며, 원소 주기율표 상 가장 왼쪽 상단에 위치해 있습니다. 수소는 우주 공간, 물질의 분해, 전기 에너지와 같은 분야에서 주로 사용되고 있습니다.

[PAGE_NUM 2] 수소는 여러 산업 분야에서 다양하게 활용됩니다. 수소는 연료 전지 기술 내에서 전기를 생산하는 데 사용되며, 수소 동력차와 같은 차량에서도 사용됩니다. 또한 수소는 금속 가공 산업과 같이 높은 열이 필요한 산업 분야에서도 사용됩니다. 이러한 수소의 활용은 점차 다양해지고 있습니다.

[PAGE_NUM 3] 환경 문제는 전 세계적으로 논란이 되고 있는 주제 중 하나입니다. 수소 자동차는 친환경 에너지원으로 알려져 있는데, 이는 지구 환경 문제를 일으키는 대기오염을 줄이는 데 큰 역할을 할 수 있습니다. 이러한 수소 자동차는 미래의 스마트한 모빌리티 시스템을 구축하는 데 큰 역할을 할 것으로 보입니다.

[PAGE_NUM 4] 일론 머스크가 설립한 스페이스X는 화성 착륙을 위해 친환경적인 수소 추진 시스템을 개발 중에 있습니다. 이러한 수소 기술은 지속 가능한 에너지 이용에 대한 대안으로 각광받고 있습니다. 미래는 수소 기술로 향하고 있기에, 수소 기술의 발전은 미래에 대한 기대감을 불러일으키고 있습니다.

[PAGE_NUM 5] 이상으로 수소 에너지의 기초적인 내용과 수소의 활용, 친환경 에너지원으로써의 수소 자동차, 미래 친환경 에너지 기술에 대해 알아보았습니다. 감사합니다.
